In [1]:
!git clone https://github.com/TheMrGhostman/Semi-supervised-Learning-with-VAE.git

fatal: destination path 'Semi-supervised-Learning-with-VAE' already exists and is not an empty directory.


In [0]:
import os 
os.chdir("Semi-supervised-Learning-with-VAE/")

In [1]:
import numpy as np
import pandas as pd
import os 
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from utils.inference import Trainer
import utils.models as m
import utils.datasets as d
from utils.layers import *

# Data

In [2]:
X = np.vstack((np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt1.npy"),
               np.load("data/sequenced_data_for_VAE_length-160_stride-10_pt2.npy")))
y = np.load("data/sequenced_data_for_VAE_length-160_stride-10_targets.npy")

In [3]:
X_train, X_test, y_train, y_test = train_test_split(np.array(X), np.array(y), test_size=0.2, random_state=666)

In [4]:
scaler = RobustScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=666)


In [6]:
train = d.H_alphaSequences(X_train, y_train)
valid = d.H_alphaSequences(X_valid, y_valid)
test = d.H_alphaSequences(X_test, y_test)

In [7]:
train_loader = torch.utils.data.DataLoader(dataset = train, batch_size=512, shuffle=True)
valid_loader = torch.utils.data.DataLoader(dataset = valid, batch_size=512, shuffle=True)

In [28]:
VAE = torch.load("models_and_losses/DeepLSTM_VAE_NLL_440ep_hidden-128_lr-{1e-3, 1e-4}_bs-300.pt")

# Freeze and Unfreeze

In [29]:
class RNN_Encoder_Classifier(nn.Module):
    def __init__(self, model, clf):
        super(RNN_Encoder_Classifier, self).__init__()
        self.model = model
        self.clf = clf
        self.frozen_encoder = False

    def freeze_encoder(self):
        for param in self.model.parameters():
            param.requires_grad = False
        self.frozen_encoder = True
        print("Encoder frozen.")

    def unfreeze_encoder(self):
        for param in self.model.parameters():
            param.requires_grad = True
        self.frozen_encoder = False
        print("Encoder unfrozen.")
    
    def forward(self, X):
        Z, mu, sigma = self.model.encoder(X)
        return self.clf(Z)

In [30]:
DNN1 = nn.Sequential(
            nn.Linear(in_features=15, out_features=128),
            nn.Sigmoid(),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=128, out_features=128),
            nn.Sigmoid(),
            nn.Dropout(p=0.3),
            nn.Linear(in_features=128, out_features=4),
)

In [31]:
model = RNN_Encoder_Classifier(model=VAE,clf= DNN1)

In [32]:
model.freeze_encoder()

Encoder frozen.


# training loop

In [17]:
optimizer= torch.optim.Adam(model.parameters(), lr=1e-2)

In [34]:
m1 = Trainer(
        model=model,
        optimizer=optimizer,
        loss_function=nn.CrossEntropyLoss(),
        scheduler=torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[20], gamma=0.1),
        tensorboard=True,
        model_name="DNN_on_LSTM-VAE_sampling_latent_space_UNFREEZE_lr-{1e-2,1e-3}_epochs-512",
        device=torch.device("cpu"),
        verbose=True
        )

AttributeError: 'LSTM' object has no attribute '_flat_weights_names'